In [1]:
import sys
import os
sys.path.insert(0, '../')

import torch
import torch.nn as nn
from functions import get_loader

from ae_functions import get_latent_dataset
from ae_models import simpleCAE, improvedCAE

from mlp_models import simpleMLP, improvedMLP
from mlp_functions import train_mlp, validate_mlp

In [6]:
DEVICE = ""
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("CUDA is available. Using GPU...")
else:
    DEVICE = torch.device("cpu")
    print("CUDA is not available. Using CPU...")

BASE_PATH = 'C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/fusion'

# Define the tool names and actions
TOOL_NAMES = ['hook', 'ruler', 'spatula', 'sshot']
ACTIONS = ['left_to_right', 'pull', 'push', 'right_to_left']

# All available object names
OBJECTS = ['0_woodenCube', '1_pearToy', '2_yogurtYellowbottle', '3_cowToy', '4_tennisBallYellowGreen',
            '5_blackCoinbag', '6_lemonSodaCan', '7_peperoneGreenToy', '8_boxEgg','9_pumpkinToy',
            '10_tomatoCan', '11_boxMilk', '12_containerNuts', '13_cornCob', '14_yellowFruitToy',
            '15_bottleNailPolisher', '16_boxRealSense', '17_clampOrange', '18_greenRectangleToy', '19_ketchupToy']

sensor_color = "color"
sensor_left = "icub_left"
sensor_right = "icub_right"
sensor_depth = "depthcolormap"

BATCH_SIZE = 8
NUM_EPOCHS = 3
LR_RATE = 1e-3
SENSOR = "color"

train_loader = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_color, "training", batch_size=BATCH_SIZE)
val_loader = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_color, "validation", batch_size=BATCH_SIZE)
test_loader = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_color, "testing", batch_size=BATCH_SIZE)

CUDA is available. Using GPU...


In [7]:
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"
weight_name = "simple/simple_cae_ne3_b8_color.pth"
trained_cae = simpleCAE().to(DEVICE)
trained_cae.load_state_dict(torch.load(model_path+weight_name))

# Extract features from the train and validation sets
train_dataset = get_latent_dataset(trained_cae, test_loader, label=1, add_noise=False, is_depth=False, device=DEVICE)
val_dataset = get_latent_dataset(trained_cae, val_loader, label=1, add_noise=False, is_depth=False, device=DEVICE)

input_dim = train_dataset[:][0].size(1)
output_dim = 4 

# Initialize a new MLP model with the same architecture
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
weight_name = "simple/simple_mlp_ne3_b8_color_action.pth"
mlp_loaded = simpleMLP(input_dim, output_dim)  # Replace 'input_dim' and 'output_dim' with appropriate values
mlp_loaded.load_state_dict(torch.load(model_path+weight_name))
mlp_loaded = mlp_loaded.to('cuda')

# Create DataLoaders for the extracted features
mlp_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
mlp_val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize
# mlp = simpleMLP(input_dim, output_dim).to(DEVICE)
# mlp_optimizer = torch.optim.Adam(mlp.parameters(), lr=LR_RATE)
mlp_lossfunction = nn.CrossEntropyLoss() 

save_path = f"C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/models/runs/results/improvedCAEimprovedMLPTool/{SENSOR}_B{BATCH_SIZE}_NE{NUM_EPOCHS}_LR{LR_RATE}/"
os.makedirs(save_path, exist_ok=True)

In [8]:
validate_mlp(mlp_loaded, mlp_lossfunction, mlp_val_loader, 1, DEVICE, save_dir=None)

Validation Loss: 0.0294
Validation Accuracy: 0.9281
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       160
           1       0.91      0.90      0.91       160
           2       0.88      0.99      0.94       160
           3       0.99      0.92      0.95       160

    accuracy                           0.93       640
   macro avg       0.93      0.93      0.93       640
weighted avg       0.93      0.93      0.93       640

